In [1]:
from header import *
import scipy.constants as cnst

In [2]:
# --- constants --- 

#masses of nuclear particles:
        
#time unit in seconds 
T_unit=10**(-15)
L_unit=10**(-12)
E_unit=cnst.eV
M_unit=E_unit*T_unit**2/L_unit**2

assert(M_unit==E_unit*T_unit**2/L_unit**2)

#mass of the alpha particle
m1=(cnst.physical_constants['alpha particle mass'][0])/M_unit

#mass of the muon
m2=(cnst.physical_constants['muon mass'][0])/M_unit

#mass of the electron
m3=cnst.m_e/M_unit

m2=m3
m1=m3

#hbar
hbar=cnst.hbar/(E_unit*T_unit)

#elementry charge
e=cnst.e

#vacume permiativity
epsilon0=(cnst.physical_constants['vacuum electric permittivity'][0])*L_unit*E_unit



In [3]:
#prefactor
def pref(lambdas):
    return (
        lambdas[0]/np.sqrt(lambdas[3]*lambdas[6])
        *lambdas[1]/np.sqrt(lambdas[4]*lambdas[7])
        *lambdas[2]/np.sqrt(lambdas[5]*lambdas[8])
        )

#messy norm term
def mnt(x,y,z): 
    return (x**2*(y+z)+y**2*(x+z)+z**2*(x+y)+x*y*z)

def I110(lambdas):

    I=(
        0.5
        *pref(lambdas)
        *((2*lambdas[0]**2 + lambdas[0]*lambdas[1] + lambdas[0]*lambdas[2] + lambdas[1]*lambdas[2])
        /(
            np.sqrt(mnt(lambdas[3],lambdas[4],lambdas[5]))
            *np.sqrt(mnt(lambdas[6],lambdas[7],lambdas[8])) 
        ))
        )
    
    return I

def I111(lambdas):

    I=(
        pref(lambdas)
        *(mnt(lambdas[0],lambdas[1],lambdas[2])
        /(
            np.sqrt(mnt(lambdas[3],lambdas[4],lambdas[5]))
            *np.sqrt(mnt(lambdas[6],lambdas[7],lambdas[8])) 
        ))
        )
    return I

def I210(lambdas):
        
    I=(
        pref(lambdas)
        *((lambdas[0]**2*(3*lambdas[0]+2*lambdas[1]+lambdas[2])+lambdas[1]**2*(lambdas[0]+lambdas[2]) + lambdas[0]*lambdas[1]*lambdas[2])
        /(
            np.sqrt(mnt(lambdas[3],lambdas[4],lambdas[5]))
            *np.sqrt(mnt(lambdas[6],lambdas[7],lambdas[8])) 
        ))
        )
    
    return I

def I300(lambdas):

    I=(
        3
        *pref(lambdas)
        *(lambdas[0]**2+lambdas[1]**2)/np.sqrt(mnt(lambdas[3],lambdas[4],lambdas[5]))
        *(lambdas[0]+lambdas[1])/np.sqrt(mnt(lambdas[6],lambdas[7],lambdas[8]))
        )

    return I

In [4]:
def N_func(i,j,alphas,betas,gammas):
    lambdas=[
        1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(betas[i])+betas[j]),1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(gammas[i])+gammas[j]),  1/(np.conjugate(betas[i])+betas[j]+np.conjugate(gammas[i])+gammas[j]),  
        0.5/(np.real(alphas[i])+np.real(betas[i])), 0.5/(np.real(alphas[i])+np.real(gammas[i])), 0.5/(np.real(betas[i])+np.real(gammas[i])),
         0.5/(np.real(alphas[j])+np.real(betas[j])), 0.5/(np.real(alphas[j])+np.real(gammas[j])), 0.5/(np.real(betas[j])+np.real(gammas[j]))
        ]
    I=I111(lambdas)
    
    return I

In [5]:
def order(first,second,third,lambdas):
    return [lambdas[first+second-1],lambdas[first+third-1],lambdas[second+third-1],
            lambdas[3],lambdas[4],lambdas[5],
            lambdas[6],lambdas[7],lambdas[8]
            ]

In [6]:

def H_func(i,j,alphas,betas,gammas):

	lambdas=[
		1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(betas[i])+betas[j]),1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(gammas[i])+gammas[j]),  1/(np.conjugate(betas[i])+betas[j]+np.conjugate(gammas[i])+gammas[j]),  
		0.5/(np.real(alphas[i])+np.real(betas[i])), 0.5/(np.real(alphas[i])+np.real(gammas[i])), 0.5/(np.real(betas[i])+np.real(gammas[i])),
		0.5/(np.real(alphas[j])+np.real(betas[j])), 0.5/(np.real(alphas[j])+np.real(gammas[j])), 0.5/(np.real(betas[j])+np.real(gammas[j]))
		]

	T=-0.5*hbar**2*(
			((alphas[j]**2+betas[j]**2)/m1  +  (alphas[j]**2+gammas[j]**2)/m2  +  (betas[j]**2+gammas[j]**2)/m3)*I111(lambdas)
			
			-2*alphas[j]*(1/m1+1/m2)*I110(order(1,2,0,lambdas))
			-2*betas[j]*(1/m1+1/m3)*I110(order(0,2,1,lambdas))  
			-2*gammas[j]*(1/m2+1/m3)*I110(lambdas) 
				
			+alphas[j]*betas[j]/m1*(I210(order(0,2,1,lambdas))+I210(order(1,2,0,lambdas))-I300(order(2,0,1,lambdas)))
			+alphas[j]*gammas[j]/m2*(I210(lambdas)+I210(order(2,1,0,lambdas))-I300(order(1,0,2,lambdas)))
			+betas[j]*gammas[j]/m3*(I210(order(1,0,2,lambdas))+I210(order(2,0,1,lambdas))-I300(lambdas))
			)
	
	V=(e**2/(4*np.pi*epsilon0))*(-2*I110(order(1,2,0,lambdas))-2*I110(order(0,2,1,lambdas))+I110(lambdas))

	print(T,V)


In [7]:
p=np.array([[1,4],
    [2,5],
    [3,6]])
H_func(0,1,*p)
H_func(1,0,*p)

1720127.9364174297 -9722.825629034796
1720127.9364174288 -9722.825629034796


In [8]:
alphas=[1,4]
betas=[2,5]
gammas=[3,6]
i=0
j=1
lambdas=[
		1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(betas[i])+betas[j]),1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(gammas[i])+gammas[j]),  1/(np.conjugate(betas[i])+betas[j]+np.conjugate(gammas[i])+gammas[j]),  
		0.5/(np.real(alphas[i])+np.real(betas[i])), 0.5/(np.real(alphas[i])+np.real(gammas[i])), 0.5/(np.real(betas[i])+np.real(gammas[i])),
		0.5/(np.real(alphas[j])+np.real(betas[j])), 0.5/(np.real(alphas[j])+np.real(gammas[j])), 0.5/(np.real(betas[j])+np.real(gammas[j]))
		]

print("I111",I111(lambdas))
print()
print("I110",I110(lambdas))
print("I101",I110(order(0,2,1,lambdas)))
print("I011",I110(order(1,2,0,lambdas)))
print()
print("I210",I210(lambdas))
print("I201",I210(order(0,2,1,lambdas)))
print("I021",I210(order(1,2,0,lambdas)))
print("I012",I210(order(2,1,0,lambdas)))
print("I120",I210(order(1,0,2,lambdas)))
print("I102",I210(order(2,0,1,lambdas)))
print()
print("I300",I300(lambdas))
print("I030",I300(order(1,0,2,lambdas)))
print("I003",I300(order(2,0,1,lambdas)))

I111 0.5186279384097199

I110 2.8873321483394934
I101 2.5268489029285597
I011 2.2928814119166567

I210 0.8370836900648111
I201 0.6974459493973891
I021 0.5800444048003446
I012 0.5345507259924746
I120 0.7976558350979902
I102 0.6129576887542018

I300 1.0943241378546185
I030 0.9288292756606824
I003 0.7081694594021013


In [9]:
dell1=(alphas[j]**2+betas[j]**2)*I111(lambdas)-2*alphas[j]*I110(order(1,2,0,lambdas))-2*betas[j]*I110(order(0,2,1,lambdas))  +alphas[j]*betas[j]*(I210(order(0,2,1,lambdas))+I210(order(2,0,1,lambdas))-I300(order(2,0,1,lambdas)))
print(dell1)
print((alphas[j]**2+betas[j]**2)*I111(lambdas),-2*alphas[j]*I110(order(1,2,0,lambdas))-2*betas[j]*I110(order(0,2,1,lambdas)),+alphas[j]*betas[j]*(I210(order(0,2,1,lambdas))+I210(order(2,0,1,lambdas))-I300(order(2,0,1,lambdas))))

-10.303111274830544
21.263745474798515 -43.61154032461885 12.044683574989792


In [10]:
print(alphas[j]*betas[j]*(I210(order(0,2,1,lambdas))+I210(order(2,0,1,lambdas))-I300(order(2,0,1,lambdas))))
print(alphas[j]*betas[j],I210(order(0,2,1,lambdas)),I210(order(2,0,1,lambdas)),-I300(order(2,0,1,lambdas)))

12.044683574989792
20 0.6974459493973891 0.6129576887542018 -0.7081694594021013


In [11]:

i=1
j=0
lambdas=[
		1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(betas[i])+betas[j]),1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(gammas[i])+gammas[j]),  1/(np.conjugate(betas[i])+betas[j]+np.conjugate(gammas[i])+gammas[j]),  
		0.5/(np.real(alphas[i])+np.real(betas[i])), 0.5/(np.real(alphas[i])+np.real(gammas[i])), 0.5/(np.real(betas[i])+np.real(gammas[i])),
		0.5/(np.real(alphas[j])+np.real(betas[j])), 0.5/(np.real(alphas[j])+np.real(gammas[j])), 0.5/(np.real(betas[j])+np.real(gammas[j]))
		]

print("I111",I111(lambdas))
print()
print("I110",I110(lambdas))
print("I101",I110(order(0,2,1,lambdas)))
print("I011",I110(order(1,2,0,lambdas)))
print()
print("I210",I210(lambdas))
print("I201",I210(order(0,2,1,lambdas)))
print("I021",I210(order(1,2,0,lambdas)))
print("I012",I210(order(2,1,0,lambdas)))
print("I120",I210(order(1,0,2,lambdas)))
print("I102",I210(order(2,0,1,lambdas)))
print()
print("I300",I300(lambdas))
print("I030",I300(order(1,0,2,lambdas)))
print("I003",I300(order(2,0,1,lambdas)))

I111 0.5186279384097199

I110 2.8873321483394934
I101 2.5268489029285597
I011 2.2928814119166567

I210 0.8370836900648111
I201 0.6974459493973891
I021 0.5800444048003446
I012 0.5345507259924746
I120 0.7976558350979902
I102 0.6129576887542018

I300 1.0943241378546185
I030 0.9288292756606824
I003 0.7081694594021013


In [12]:
dell1=(alphas[j]**2+betas[j]**2)*I111(lambdas)-2*alphas[j]*I110(order(1,2,0,lambdas))-2*betas[j]*I110(order(0,2,1,lambdas))  +alphas[j]*betas[j]*(I210(order(0,2,1,lambdas))+I210(order(2,0,1,lambdas))-I300(order(2,0,1,lambdas)))
print(dell1)
print((alphas[j]**2+betas[j]**2)*I111(lambdas),-2*alphas[j]*I110(order(1,2,0,lambdas))-2*betas[j]*I110(order(0,2,1,lambdas)),+alphas[j]*betas[j]*(I210(order(0,2,1,lambdas))+I210(order(2,0,1,lambdas))-I300(order(2,0,1,lambdas))))

-10.895550385999975
2.5931396920485996 -14.693158435547552 1.2044683574989792


In [13]:
print(alphas[j]*betas[j]*(I210(order(0,2,1,lambdas))+I210(order(2,0,1,lambdas))-I300(order(2,0,1,lambdas))))
print(alphas[j]*betas[j],I210(order(0,2,1,lambdas)),I210(order(2,0,1,lambdas)),-I300(order(2,0,1,lambdas)))

1.2044683574989792
2 0.6974459493973891 0.6129576887542018 -0.7081694594021013
